<img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAG8AAABuCAYAAAApmU3FAAAEtmlUWHRYTUw6Y29tLmFkb2JlLnht
cAAAAAAAPD94cGFja2V0IGJlZ2luPSLvu78iIGlkPSJXNU0wTXBDZWhpSHpyZVN6TlRjemtjOWQi
Pz4KPHg6eG1wbWV0YSB4bWxuczp4PSJhZG9iZTpuczptZXRhLyIgeDp4bXB0az0iWE1QIENvcmUg
NS41LjAiPgogPHJkZjpSREYgeG1sbnM6cmRmPSJodHRwOi8vd3d3LnczLm9yZy8xOTk5LzAyLzIy
LXJkZi1zeW50YXgtbnMjIj4KICA8cmRmOkRlc2NyaXB0aW9uIHJkZjphYm91dD0iIgogICAgeG1s
bnM6ZXhpZj0iaHR0cDovL25zLmFkb2JlLmNvbS9leGlmLzEuMC8iCiAgICB4bWxuczp0aWZmPSJo
dHRwOi8vbnMuYWRvYmUuY29tL3RpZmYvMS4wLyIKICAgIHhtbG5zOnBob3Rvc2hvcD0iaHR0cDov
L25zLmFkb2JlLmNvbS9waG90b3Nob3AvMS4wLyIKICAgIHhtbG5zOnhtcD0iaHR0cDovL25zLmFk
b2JlLmNvbS94YXAvMS4wLyIKICAgIHhtbG5zOnhtcE1NPSJodHRwOi8vbnMuYWRvYmUuY29tL3hh
cC8xLjAvbW0vIgogICAgeG1sbnM6c3RFdnQ9Imh0dHA6Ly9ucy5hZG9iZS5jb20veGFwLzEuMC9z
VHlwZS9SZXNvdXJjZUV2ZW50IyIKICAgZXhpZjpQaXhlbFhEaW1lbnNpb249IjExMSIKICAgZXhp
ZjpQaXhlbFlEaW1lbnNpb249IjExMCIKICAgZXhpZjpDb2xvclNwYWNlPSIxIgogICB0aWZmOklt
YWdlV2lkdGg9IjExMSIKICAgdGlmZjpJbWFnZUxlbmd0aD0iMTEwIgogICB0aWZmOlJlc29sdXRp
b25Vbml0PSIyIgogICB0aWZmOlhSZXNvbHV0aW9uPSI3Mi4wIgogICB0aWZmOllSZXNvbHV0aW9u
PSI3Mi4wIgogICBwaG90b3Nob3A6Q29sb3JNb2RlPSIzIgogICBwaG90b3Nob3A6SUNDUHJvZmls
ZT0ic1JHQiBJRUM2MTk2Ni0yLjEiCiAgIHhtcDpNb2RpZnlEYXRlPSIyMDIwLTAyLTEzVDE2OjQ4
OjM2KzAxOjAwIgogICB4bXA6TWV0YWRhdGFEYXRlPSIyMDIwLTAyLTEzVDE2OjQ4OjM2KzAxOjAw
Ij4KICAgPHhtcE1NOkhpc3Rvcnk+CiAgICA8cmRmOlNlcT4KICAgICA8cmRmOmxpCiAgICAgIHN0
RXZ0OmFjdGlvbj0icHJvZHVjZWQiCiAgICAgIHN0RXZ0OnNvZnR3YXJlQWdlbnQ9IkFmZmluaXR5
IERlc2lnbmVyIDEuNy4zIgogICAgICBzdEV2dDp3aGVuPSIyMDIwLTAyLTEzVDE2OjQ4OjM2KzAx
OjAwIi8+CiAgICA8L3JkZjpTZXE+CiAgIDwveG1wTU06SGlzdG9yeT4KICA8L3JkZjpEZXNjcmlw
dGlvbj4KIDwvcmRmOlJERj4KPC94OnhtcG1ldGE+Cjw/eHBhY2tldCBlbmQ9InIiPz63ZYCCAAAB
gWlDQ1BzUkdCIElFQzYxOTY2LTIuMQAAKJF1kc8rRFEUxz9miDCNYmFh8dKwQoMSG4uZ/Cosxii/
Nm+eeTNq3szrvZk02SpbRYmNXwv+ArbKWikiJSsLa2KDnvM8NZI5t3PP537vPad7zwVfPKMZdmUY
jGzeio1ElJnZOaX6ER8BgoQJqJptTkwNxylrbzdUuPGq061V/ty/VreYtDWoqBEe1EwrLzwqPL6c
N13eFG7S0uqi8LFwhyUXFL529YTHTy6nPP5w2YrHouBrEFZSvzjxi7W0ZQjLywkZmYL2cx/3JfXJ
7PSUxFbxFmxijBBBYYwhovTRzYDMfXTSQ5esKJMf/s6fJCe5mswmRSyWSJEmT4eoBamelKiLnpSR
oej2/29fbb23x6teH4GqB8d5aYPqDfhcd5z3fcf5PAD/PZxlS/m5Peh/FX29pIV2IbgKJ+clLbEF
p2vQfGeqlvot+cV9ug7PRxCYhcZLqJ33evazz+EtxFfkqy5gewfa5Xxw4QtXymffaj9hFAAAAAlw
SFlzAAALEwAACxMBAJqcGAAAIABJREFUeJzlfXu0b1V13jfX/l2UkAAXiq/EWkg0RqUaBO5BEB+A
WKtG6qtWeZnRtCYmgThUvJfRdDQOBAtc2mSEGJuhJn2oNcPwGFZSgmmAG84JAgKS4QPIQ2OVIgIR
kbP3mv1jrW+ub+177uUROIfENca5557923vttefzm4+1f4a/R2PpfX+wCfB/6o4XmuHpDuwDYG94
3gfue8PSPoDXY74PAAPsO4DfBeAuuH8HwF0wuwue7wLwHbjfCUvXA37j8gfflDfy+R7usI1ewK7G
0taLDDYcDM+HAX4oYIcC+WA4ngAA7hPMBgAOdwBe6W78fwLgdTYHcgZMH9fkcwDu9wC+A+5XAX4l
HCvL5/2r+x/r5/y7jMcd85a2XbwZ7qcC+LcAngnPgA0AMjxnmBWmuDvgXp7AEpAn8HEsJXge42+4
F4bWa8v59XOzem0dZuV4Hh+ApRUAvwXg48vnvkVOenyMxw3zlrZd/NNw/BLS8GZ43hNAIao1BrhP
hfAuGgPv/7bUtJDHybjQSqsM9TI/f8OLMpoV5lpo71fhfjaA310+762rjx0VHt7YcOYtbb14APw/
AHYG4Ina4NMIS9Q4B+AwS4WBYRKt0p4MyeUnLapmoTGHn9NUUlvNeoZ3x0RjPQPufwXg/cvnv+3D
60qkXYwNZd7Stkuejjz9D6ThyJD+PAFpCLPmnmFpUc1gHe5ACoIKgyB/k1FJTKM3xtJ02tBrKjUw
tDNXjfV2rk8fhg0/v3z+22RR6z82jHlL2y55CoDrAH8qcg/y3Au4MBt2YlrxZ5XBXnygT+MMjPjO
ZnH+GdD8GzXWrAKbVE8VkFNQEYJkZp+F+xuXz3/b3z5qRHmYY0OYt7T1ogFp+CPk6SWFeFMjWIwC
Skx8XPkbDazU84KwlprWzU1lZxpz0dz4HLO50LRVGRpgZrX8TmkF03Tk8gUnb4gGzim2PsPSL8Hz
S4pvQTFl1gjpOQOwWw32e3D/IOB/6PD7GuKk9Nd/lPHho1zCh6F95t4+h/faFXPItY6i5TzXpzKf
GTCtHg7kdz0WJHooY4M07+IvwfCsnTSgEPT7gJ0J+PnXfOCEUIstZ/z+QfD8EcCODlNIwpMRadH+
n6uvUpNM9NihTAkbwr+hHR8W5bjOE/cbAM/fA9LBy9tPvPWxp1w/1l3zls689KcAPKuBAxS/FmbQ
zrjmA687VxkHAMtnv/42IP0zAF+J2DotGrRPi+a7GL8V0ybocaomcyi/PUtIgP4ar2FdzsA0NoZb
qnOGOd8Tnn/lMSTZLsf6m808Pa1QvHIgZ1g1AA6/BZb+064uXT7n9ffB/Vfi+jw1rXA035THerwy
K5AoKuMkflQtpp9k0E4/aalqoBemamhS5njto0ylhzTWn3nue9e0SDsEr+gyXXnNWa/1XV8MwOxK
5MnDjwFNU+bprzTIMQ0hNPbj6akxK9bnTStpHQInTW2ePP7YltM++pOPiB5/h7H+zFNixqGB0P7r
D3b58jlvuBuGv21xW2rapXDeJ0l7iX8LBlHbqKFjM5VqGTT/GdfIOQ3wbH5kBHnkYyM0L6TbyYD2
4Qse7PIt7/mfB8GGHwngEUBCgErn9/gZ/5uFSWjnpEHSYdbmVpNr1pAnNd8oPHmvh0WHR2FsgOYl
78ICAD6twvMq3P2YLe/91NN2P4Gf2IJqAQ5e0WZ3qve/89SHCpGoBpoPo7kU5jOjA2tpNxfGlucS
O74+YwM0L7vnqeUjCenLUvaBpd9Zet+nn7DWpVve+6kluG9rlQY0eM+Umqmv85YWAyT1RWZULVNf
F0E7E9XU6LGZyC75zZBk/YsOiwc/5VEelqqgGzyPsKH4O28EeKXDPr/lvZ86HcCO5XPe8N0t7/nk
gYCfBPdtSMOmIKrGekDlUZ5pHHOSqiVDQ6oMDyLXSbSp9b6aI63rLxUOF3xkcKx/HXfdmec+eanM
5JK7nCTOipPwXAB/CEt5y3s+eS8s7dOZyqzpNEkYIwMZDcQwpZXrZ/Rb02rVzkFCi6mFEWloZjUL
uDEAuRy3xDiPHFz/fMeGABavIMMJ77uaGhoC9Jxqa0Pzb6yIa+qr0yoBIG0elOpCnZtZE0B8GEow
DrRrVDOT3CeY5pJEX39Srvsdzcwh5ibCBE1XdflGia20YFoub+dS0wL+s56X+nlMtDLCCxEGZZiW
hSRn6vF3ERCnP1znse7Mcy+9C5TUIEQgwrFnQjkLkdmofStdgB3aKNobGj37LO4nzCK6JIiJFFgV
FmpdvcYkK2NpKBmieXJ7HcYGmM2pl9ZgkPgoVhriGgnC5y0Olsq10aOC5hej4sCAWq5vGeimpXls
xVqGFU7EWgcjAiuFYneHw2vMur5jA0pC5s1PqUZQuxRF1thK+1aiilCD7TxJ2CDEZqI6j81carmo
TIZoYAKkKmFtroj35uZVwQpm/1+fsQGhgt0F+EoLkGexVaC+DJiEBABCe9pkaFpnpUhqYu48HwBL
B3btFWFGJTfJtczBEOF/lK5QUHIaaqhg1exjQ3zeujNv+ezX7wCwZT3uteVd//3nkFc/1CHNAC1j
0zr1i+5NeCAJ7yo0BgNbED2P5TfyRijeBlXS12uwvcIgOc+ZSQaaKaU/Y1amlXzanEY5GMWkAj8Y
ZnM9R2vZqwckrUWTnTMwbEIgWogZlbjOvSYVKsii6fRJTXU9/Uuv2gTg2e2AmHqa4HLsfqT0NfvJ
z3zvkTzeI2be0taL/gmAFwD4EcD38pz3CB8QNbBqTtT5R85Q4qkOtksdTf2Ipq26Vr00+5znJ8Dz
i5qpBKKaEAG/oEzXBHWZ130M80gNNjcgLcpzaryow+zpyPnGFn9KImKo2j1NwUS/5fhvYpo+gyH9
N6T0OXv2Zx+SA31YzFvadslzkafTYDjO8/QMmDVj0bXIsebFzEaF85GqYkklz9oQagzHdJYy2q0x
wazMC0k6M/Ofs/ipSoMk3oHCFOkzHuzDDc+rpVBAgJInwAWgUHNZlVBBIzN530kySWPtPOMYRwB4
MobhVOR8Kgxf8FuO/1l7zmWffzB+PCTmLW296KcA/Cry+EZ3T5Y2ITqVE3srrVTEA9ExgwE0xqnk
A/DKAEWTXe1NiqkkUszhHbEjRLCESDzrCHOVmxWIuayhyDzV7rRZ0M/1dTW/mWWY34u1Rs0WJUky
hPmsP0Xgng9g2W865gKkdKY993/vcrPLgwKWpW2XnAazm+H+ZqRFsnkPiDNRO7RUF9vRI+huaC0W
O60CzhhsjQxFxFGM8yDxWDU9adGIaJU48WQsrgoTtOJe40P2gFptKrKUyrF54G1SPO7mUZ/Ke6fW
tMTPh6Ektdlvw2ec6ArQaJB9APAumH3Cbz52l3XC3TJvaetF5wDYDs8p6mXOHTq5Pmiu8HmqWZOp
ma2utQBlkWmoi2fCl5IM0VhJWelxDs1fMl6DVWZWjexKPgI+wtQVs12sKBkx1tuKH2Uuk+uf+1lp
pIoRjVAtPsS4Kv5fsMBClGGamvkv87wW7h/aFX/WNJtLWy9eAPnDMDslNElQGptenf6LD2KMjypx
tR1PC6gBSKb2kNTSaSzoL1Jn1KxZlgVDIcA0NkKQwMEsxmqV4SRK5EfJoLKZxYZF067a09btTCKo
0awQ5LY6VGiGoWlgqpYkz0KRaSqfASJ0BsB/1r/wsm/a8z+3bc6ntTXP0llAOqXFRnG8oLA8VdNS
Vh3mJczpPL1Ua2eay9SgmAum1NHkkNH0ozQt0XPJuI0mVh6c9yUz6V+8AhptgIo8p7d1eC4+XEtO
XaUjt7/Dt5Oq2maB8jyhmXIvjhAKNFNKC1TQ6lb/wste+qDMW9p26cHw6fQwWyGZU0VcxVx2N2cQ
PG/0mZswhda6BYvt7juVdupc02qbs0tAo80BZaBIddTrhGA2NEJV0112IjUA07RNLEeUoxTQ9DFe
WS/jxWo1hmrgOsBEoAVhNh8jiaYygZB+bbfMW9p2scHzhQAWXFTEbZVAJZMucVkEwd7MU0dINGaE
mVMgUn0XyyoEHurjorIdpgTR/cw+mLgZeq1QjYxTKqGs7PcrlkRQL58nykv1+V3vMbVzZ7ucwqpM
UzmdoYIKb3QQzEDPsEYpqtDiKL/5uFfuknlwfzs8H9mYUbuZxdwZWwd0oXyIIK7U4whcaMLU9HRz
eDOjMZcjup6VgXwgvQcRZAT6aASJ0hAJS186iSZhdo6uR4QjGKv+G3IO+nZ4Pi5Nfao0YDVfa4cE
LDxOS8C5p/H9azJvaetF+8HzOVGnmka0doXysNwPHg+VhjUemGiTU1szF3NU2ZlVl9jMW0tCwOc6
r2p7hAre5gsi8wmHdj4tQzAacg1ijYY1BDJMKoShJmHL0Cbi/MmaeeZzMDTIUwUt1LihziXuIwlj
i+97oX/hZSfsxDwAr0da7A+vm/bVRDWSCFDxtkitXishwyyuUadTgpJwFIaYYwZCyCyb/9Asz7Sb
Bd9KYBZPkYYGGQQYmACyZjabiQ2hCCwgXdl5ujMIRUtBk06XQS0jIEvCKPrJCOyBTvsbuj1pDebZ
axRVlcfPwazYE6caFLtW6cBpwggWZoleMoeL73wTmu0PM6rVcGEGGRuwXvxuWIYJGDbB81iSx1Uo
zQzclWRm8Vw2LJRATUs7nyrWJExvmNK2xcvQC2wgXQfGSTatCA1oNlF9ZeVD0GQKIXmpX390itVs
ed+n9wT8WBLLp7FpWP3tObfEcOcjKqHClXlPhDhXfGcSUxupJjEt4QuGZi41YzNvoaD5oykXrbe0
6PKv7r2AEJC1JAN9GgVKgImJC6BFKWu6a+U3f+HuYJ7jhxroIAPQJ+qDJmjAJqWegdPUBIfmMw37
AnaIitIxAPYMems3sRJZQYQGw47mo8BFUmMk2CYzuz0ALueiI3znc9RfMfaKdjxv6xO/2e2FMPRz
1vCnnKoMErChg3Nr7MvKhPtt/cnYjNnlSAnYJD6a9NAYOdamzEqx3qAP8HKgMe81ZS0JDm/pIfqP
eWNsgAPmLtE+VyDCDAcJQyIGQqsPEYlqMUXA7CFl9yofQs1y+EoLKbeaj6Rv8067lbo0+anN26Fe
yO96r7SogGIA4F/pGOX5yQ0zoAnjRDch1omt+cMafaH8m0E7Ndi9MG9p60UGz6/2PNYgvEqsEpSp
LDJTt0JRGud+kAuIWlldTCR1yUlv53BOoAGhPJZjBACRlSdjUyM2rwPi/OLnNOCmkJBpMxVRJmpI
MNV10OfHtjAHHB/tJ7FDIpaLa9GupwmdRrFimIVD4mLCDdWflI7y64/elAAcbGnT07pMAaVOW73Z
EtehTTWHEJ8DMWEm51eNC5RIAqqJFOK6EI+f5zESyEHI3cRlDMDL+1x0syUagTSeUs1IA0pTkyLa
qlFtrdevXPgLl80E4NAODSexPjS7QwVvAzu7s1gSQdFAu74h2L3gfngC8AyP+MmDwxEuKEgwiZnI
oFhY7n9Dru2y8MLIebqL85LIEdPVefOIYqqHnshdeGJ9bFeRqWkIQI2nCeR6w0QrwJgxrSsLATA7
u+PbLcfvA7MXt8Ba/CkZMQzSWs+YlWkwEX5eT6aPYwM9ln5iAfcnGayP39Ls5TV8OB8BWwBWzUaH
nOjkO6kUTSDQQBAUruhtQlTd1ZSF7xDtjHQa47oGcFwyQy6C4socCkggOfU3uS8ok9nRgsF1AnC/
Co5PdWTK+U0wPHEnwEJGKWPU6hDkaSot3mihvjeu2y+55wN6U5b6XJ0iHfo++sQOneXGJI3/mAmx
AQ3V5fZDn8bqt27yiLixmkma6nnch5pvZes56muvQivFZ3DNaiki1qoE5prYOa2Zm2YlbgDw6pUL
3ynEAmB2SnM3aDSdB9vamhHgSxirrznRYyzowvZPgD/JmeDtHnbmz8qCZwxV/8HfDGylKKsxEc3g
fN4u3yemMF4nlXsiaPAPlEqH50qvYq7DelDbg2FTM8XhRyqj0tBnepJaAoIGfBnA8SsXvrPFdgD8
pmNOgOFFYDhkQhemuvi8I4W2Mneeu233apoIIbn7fgnuB4APTM1R6Qy4LhNxMRGDCfoLPyPMzFPf
rxEmgMldQaJdloMIsoYc1VRHpcPLG5FomlvFm+ZbQBj3IYRGpGYeO8FySV/NUnSwDPffgOfDVi58
57c6xt1y/N5I6deDaewYMDT/FiBQ6McffemPXk9hXXDtwdj9F7DhgCKxRIaE/VRXJaS3vF0XB1VH
qy8oXdPUKKiRRQdA8fbAZs2UpiTINcGqz/W4m9ellDY9r1UNs6H4QEW1FJBAyxA0twnRbq9pwPJW
ps/B/d+tXPjOP8NaI+f/iGQ/iuw1IS0WbBiKpnVIl5ZlRKcsVgVrHAvDXOitRemU9l8grz6ptB2o
TxATGnvGrRJWu4plIT5jfqOsCINoH48HoYTp80KofqadyqGB3PJn3Y8Hg6g9VZvYHjgfLn7H7Btw
fAbApUC6fOXCd6z5dj//4nEJjv8M+M9h8mak4vmtCUxknKgIEkrlXJjOdghqIbCz0PsEjHm/BVLR
vI6ovtODtGNm4NbeLh5TbmlGJgCQmGMtnoapSwCQYelr8PwXgN0Oz7fD7K8Af6CthUAJMNkn7gGk
ir+1oXaGdY7HKbVNSLUoCn8ANvw/AN8A8KWV3/z5uXeeMe4VewD4r0h4Y6EJetPfIdrUEOQglksz
TzTnzG+yd5XghY26AJBs/wXcD2iV4txu0nG6SvicYRFbAXBrksZrWVCcB9uxWJqA4Vb4tB2eP7p8
wanf3R3BHg/Dv/iKAYZ/AeAMTNMhzS2gPN+waIzScABoTCATeYznpxqGgGCtCoFBqhEZsLRfeYWs
2RNCAl1UOfyeMqT+bUCX0KW50tTaTv8XiSztENfBcBZ8+vTy9pN7yP04HP7F4/aCpVORp9PhdlAD
EIqIKw0m8Z0BSmoBNlnNsFifOovwoTJ8IUKgKLT8f7GApW8jT3t1BVTaVUV+mrkgpI662moDFVrR
ZjosNLf6xDwBw/AJ5Omk5e0nP7Bbgt36WoPj2UjpqQD+Edw3qa8Ln7xYAKurAqYaIu2C/m5y75+R
xwAgT3vA7BmwdCA8HwjgQLg/Dcy0s0o+DDWwJt1nYEVjNEuAMVTKxVrRtC4WBaRELnMWMpDp4bL8
jgXc7wTs6bH4OVzv8p2KzrwRJikqym3xSowIrg2A/wbcf3lX2ua3n/DDcH8F3P85gFdhSE9ppRNF
sYKIV1clTuR5NF1o66af0fgt1wLpODYm0DoshkJI+u089oRG1SICuoiXp1Y1j1dEyr1ppZgO0zQa
aa3YgoicG04Ndyxg9u0mhYKGIoshMVDEdSSIossZKtQ4K2fV2g8tX3DKL67JtNtedxhS+jXk/DJ4
3iPMM32Hmhb2oOi7U0ZaAAVSQORgdcOH1hR5zWJRN4IAARY4ZwgKWpYj6AW00EJACIWAvmwcWwMu
QwKuTWNb3oda7UJfPlv2b1XNkws7rRHg0pkXQXHsPiYRyVQyXetw7l+DpXfvxLTbT9gHwFkoX4RR
uBPhgtwvbsv1oPkM9oOQoMq8ObKLG3tjqhKGWhfPVCV+FLRHLaegJ12P9UIUAizaxmegxiUqhstn
uc3lst7yHHcsANxZ8o6y6aNDSJVBXfZCCShmkk45pcKDQJxBzF9cvuDke2eMezLMrgTwzK5NgPPx
4RYJmBxRP9xUY9Ms4QnkWhKN/khNJC1EEFilXyxLlzRGAxqrouFRRKXgcK6ZBSsLateR2fVwFFwZ
zpBRhvLcerwo0h0LAN8ugEO+SEIz+J2ZRNOmSOmkoEVZHOcRphe/tGP5gpP/oGPcrT+zL+CXwf2Z
gUJpWiiB9K9Tbg8ENLMXcafJGivT1io30UcpWOEPG4KzfGbWgMJ8h5TGcXm2PiL30LrKtGkqjbWj
Jjkga8lFq8kwVt/p64Pu9q0En+4MBjHumL8ag1II1NZzmjCaqGpC9c14ob0RoH+kY9xtrxswDJdi
ys+PhyUImGfY1VxqtkQDYK41KumpgQEOJocJyghCumNimgIhVwZyz0HkQ2XOiJUpUFPzjWRaZLFQ
zaSXOQZZv5pOBTkqbFXzElABC9M1hgL9g8piOkmAcN6cXJtnNBasmgi7HzZ8sqck3oBpPLKLIwfZ
vcNANsydEHyetVDfxnP1Nz9T4k/in6PBNzXfM69+Z0oQL7PGFGZEGLsF4+u5C/nqAQbwleQlkSLm
VTW1s25MkjgTH99K8PzVDmzk3BjVDXX+ufkeGOLteZGzEwRaNPZPlrefeM9swjM6BpBZ08xEa72N
95//n0HxnFkkRgfRhSGGHq6vFRMGgEDTyHk7BURoKEya0spiwsO6uIAs5aS3+QKJkt402QYMw18m
pGEZebyvI1j7o03UESv3Gth1JgtYCH/p13b0uO11rwT8BYHq4vz6QLx1wO46V0BsIZzC/W4blyDk
WKcgOYIE9enh66oQae1xlFdakVFKM01vxT1TLeUQnHhDnJoLzt7ckJpJ9b2ct1igr9uhV9+Ulj/4
plUAV3WZFd0xoxl+QJx6bsejoCmV8ngwA/J0PfpxcixwsakndHBOhaTeN8ykCJKO+JvmGmJWvfdN
ASzQGNglJyp9Ax2KT5s34ur9wxx7M6lMKJNJfL7AAxJGwFr2SunY3+9SgDFVnq5oDEqifSqp9WZE
f3GMNlhU2oSRAJCG/9stIqVntUR/tfe6NzsyN7UgSUiuplkbc3ZiHnpBMjTwQIZGwL5mSNMj2MUi
yBECADQBoECr1lHAIheZm5nk/8NXo6BP0s1zq3bMM15lfZc05qXhisZVgeBRVYDEb/Q3JFqIaCOU
nlMWc1fHvDwd1CVdqRV8IKKxCGAZpOaWodCM/NyUdRqUZMuwmOEQPPVR4kuj3ua6T6DXPIKfeHPS
fI66lkGEkMdHdo95KGKPgkla702/+31I6Y8a89yvQ/ny22abI7OAtkDd/xYpKzKbN1KmGh8s3vDj
t/3MZjj2bcGzEE/TV3QJaq7Y+kaidGhUND0YMglqI9Pq85lJ3CYok9pIN9Jp1Ow3nzPiSgF19OFz
v6sKMBc0bUqKBAM/j2sut8N23B/MWz73LRMs/Z82GdC+uUoXK46XQEXjriTZhsiG74TgDmrghLGY
iTlSLbKSWxzHpnV8yEmkn0SMPKAAhjnhVCMDLddniFwi56wCM6kJFqao1g5KL2o5BG3Wv8dVRCIc
JJUCk1mTMUfskygmM5hXz/5YOFCF13yIMIlomhgtDLwBfYpu/XJ0oUfO+4VvzD5rBec9vQSuSRav
64k+FN6mmqbIO3b3Q2icSnI8q65bwoa2A6jXHn6mQhGJ+TrXKAham5LpUuiS3AXMiMCwMVefs8x1
Jwy/z0cL5i2f+5ZPw+y64oemxkQ6UWk/6Fr4qDnKxMhs5J0/s2QN+ub2m3NrVVmddWR/rIUF/FsZ
qyCmi9vE5ykQYtigoQQ1idLO8yaGRPO38KKh4PkboCZBx/W7bzvUazNTyaGC1hIT77Ety4EfNKgD
3M9sNTn1bWhMUCIFKqLHFScf/hPw7utf0AhkqaG6CHB5L5F0EltbCCaZk+uZ5N4hsZAsiAlheJ0w
vGNcbhkTNWls7VBfpfGfZlE81xcEKHOsjxk9NxrPn8tFcFK6CrMUY8e85fPe+r/g+erS4azZjdSY
GYvNTYIiEW3N7xEYwNF9U4sGoWqidX+CnsuHzlMzR3ywrmXQRYgg73KhZgnxBvF9LGdpBkY1Xv3R
HM1Sc8ISiL/TZ1P0Teuim0cIqjQQLzfisRHu77At13Tgode8Ms7s0lGdJnJRs4ckoQn7eWPV0m5Q
CFzMjsRt6j+AhiTVVDIpQPMSx/U2FAaa/9yuiWwImhVQwKM51KCWmN7QGAI69BmmoAFmbkPWQlMM
VFAmrqrDHNhuW665eUbEnZm3fN5b/xjwDwBoTOs2XpAp9ccnRCcyH2JuuvoAU/yo9egqJJAbSGbS
3z1Ybr6QhKT0x1rElFVaYlhIcD2188NyaIoqt/NCKyhg3uI30on5x8izyrwmAqvxnPp0BuYqjO4f
AXzrnE9rMg8Als8/cSty/p0OXWljT7xPE+hfj8jPw8ije+kOmaRARcERTIgukqdxnprnTtO8wvCM
9ooMa2YptGvs51V/C/Smscs00fzJ/+daT589jwMpH9pOou4jwikTkDQBns+2Lde83ZZW5lu2ds28
uuB/A/dLupvoj/rEWCQX1kIOs/JqqI55Qu8ABmqeDU0zgXaPJJoffnBViGZFswiK6HcDHBFooEl4
aCzavES2fI400+5AvqjCQBJInMqmJK4hsk2VftpAO8z83jRmpOE0W1p53675sxvmLZ9/4gTPbwL8
XHBbTZiuhO6Vv2ivtepeb5jl1VA6KImguRCkN1Vgopn6PBccawxhvEn/BbTYLszXDHIHUhbkGmGD
mPqowtc5dN84n4OVDsfOn2lBNWpyMyTNzwvKzAA+jsWm59mWa3b53bkcu33T7fIFp9wP4N1bTv/Y
J2DpvyCPz+/MpCR1Tb4lK76MHrLIGBbXtIQ0mlkmTJ/7A/VD04yJg5hQdnsRQDCvyevDp6HPuJDA
4wNVe8UsEq3SpLKqDjRzP9hsF2zVWAIwmzFXqxXT9BcwuxzAdjtsxy0PxrSHxDyO5e0nX7vl9N89
BEgvhftbAX8VgKdQIt3Lq33L/lMBDRHIayAKRCGSqkImAD3TFDoHEwW0kDGT9L2oBho+gWHYEeBh
sWi9MBQSZUDsfvW2VgIJIuMQskWLcRXR8hmAZqYZzxZmOdwfwDR9F8C9SOkGW1r+y4fKMB0P+QXh
y9tPygCuqD/YctrHnuh5/DEz7GlW3nhbHrq+PTaXLVaFFvmvu8l0E2UMYRglUomrCDE6r6qTV+bT
XBWze4U97/Lffvhk+fsxHvGr+ZcvOPl+AF992Be6l6B+WDTzq8XOMCVjCyOUOdos1OUE0cy4zv0P
eOwGbT5GQ512Za/2AAAGk0lEQVR2OO4ZGKH/UShOfxdZixpzAQ3QRIqL6aq5v/2HNdb96fwrr34F
3C/baRNI53/q79i5RGbl5gcJRgABQESh4HW3I093xHxd2kv8FNegIQGArg2ReV7W/SLQnmjuT7cX
XrXjsaVeP9b/62imsTl7LbQqDGeQ2qW9rOQkycDYl+C9CdXCMXAggAM7eE4NjS1WRM4AWDW3es9x
td2b3+MwocVzPSredz3JCGyE2eQGQaBVxiMGm2bmEQ2SR6uhN3PK7mOgbhARwKNF4C6LgYY0x1W5
t68RlqDX1s7k13lih5CvuxVbf+Ypk9SfsUJBItGE6WZ65hKZ+1ToD8yakrw3icyBRnNT6lNs9JXU
NveyvUtrl5C5w+dKqLDOYwMAC9NT1jSA2580Oa1EjJTS2ALmlErHlTID9Th7JSN88AZgmDXRF5Jq
Wajrz0G/b0IFjvFdtGesv+atv8/TLEPEc9REOYd5zAgnagDNrjNDM5nRGzk086iM7zrMpMkpdqJK
vbKbU2NRQ7z4lOerSd0A1dsYs6lMYgaCCFELrgos6F/4uZpLVhk08cuhQMhQvy2lfqbfIMJMUNsj
cTeGRY4LDW0bFhm32NTuP6/9rcNYf+YBaBCdYEUaf0ikaI+QGhlfREPma8CulXXdr85qNFAB0GpD
lgGOohK+imH4VeR8oB161b4YV/fGlI9DnkohNFJh6NdfKgQ/AGZTW/uGRZ/MjRqcNW2wVEQs59q+
gIoyB9m0QfOmYCVJrKjZGGvmcKgvIcAEGFZhdrQdcuU1XKkdevV3AVzuf/aiQ5HzRUjp+NB83XK8
QWNjMiwVrLXyDNC1FnK/OZFp9t7XAdJdVf+eo1c1qTw/kGdq8yxq/S8N71fGdUs+bMf3AbwdZndj
sUkajAQo/UCECu5TfZVHe/goflauRlGTCBGI/dkKSHgNzaXWzroYTyr9+jIAfU2w++/tbtl22I6/
wTRd0Xp5INmetCFmcwN8nt/XvZ4wsiJSzQYEkqOdQ7MaxUuaSumvpK/TnpfwaeIvQ+MBwO+1Q/7k
9gdfut8Qr9HXtdInr/PYAObZtyNmA1qKS7c/aexGLQlGQNJqEo8FcrWdi6xw6ZWEZEyMgOWH/fqX
7P3gS7cf7RLpIWAJbSHrNzbCbH4Znr8evfpEfFrCGdQMQpjkLTaMzIYyCYIy69BOLrO+Z5MhhrvB
84t3u+xrj0wAjgqB0op/ue/du7v+sRjrzjx7zmUO2MVdjyT7Q7gnnuYsDe0ryRTSR2c0FKrLXRgC
VOSq/nGU6reWobJv9+uO/qFdLtz9l5Hzc7rNpYCGLd94lEn1oGNj4jzDeRiG73WxlrYR6LtTmDCm
BmkQP++P0ewJMygEL9oyr2kwCozZMwH/rK8ccZBO6StHJP/8Ue9GSmfvFEMyP+r5S3b4n/75Y0ix
Ncf6Z1Pr8D9/5RkYxw/0mxGtT5kB0jAk8eFik/SPiG8EehPLlxPoFuac+1ogEwEAanvgfUjpapjd
gGl6KoAjYPbj8ebZiCnRwg/D6ba0csFjRqxdjI1j3s3HLmB2NWCHRygQucO6rD53KIyaSOiIJgKF
As3EMl/qKHOPYvKY01TgpALSNRMxeSCC1HKnfwzgWFtaXne4uUHpMcCed/mI7McC/tnQgvCD9f8B
asTfjWML4COrwmBZ0l1R2/NWr1NNBXrww7+jIq/t+DO/GUjX/wbu/3IjGAdsIPMAwA6+/F64vwZm
H44QAOjLQpF+8p7w1AYWUMlUoCWcu9hLEWZifNfQbAAXVvA5h7THq8bBrgKwZEesfPPRpcpDHxtm
NufDb3z5v4bZmQD+cSMS0IUCWkvr+lG8wfcI0tFMJzvRtD+FrRgRuM+Tzt7/EOWaZZidBc//3pZW
NrRF7XHDPADwm47ZBPeTYHYGzH5izdoc2x9iP1xq7RDUIiagqc3Zm0YpwGGCW30cINoOFZLvIaWP
w/Ov25bl+XtlNmQ8rpjH4Te+fADwZri/A8DhAPbYSdvU5AHS1ETtmyQmNOkFTTMhGNCFGLw2Kh7+
ZQC/hewftSNW7lp7xRszHpfM0+E3vOSJsHQ4zI6C+4uR8xEw7BMJaIYE829GjpeX1oCeGsgsjVbY
yeSUvg/3GwFcC7NrMY7XArjJjljx3Sxxw8bjnnnz4dcdnWA4GJZ+GsD+yHlfmG0GsBmeNwPYF47N
SGkzxtXN1UfdDUv3IOe7Ybgb7vfUdNY9MLsbwF8j588DuMkO/9PV3d3/8TT+P2x6AX5Riia7AAAA
AElFTkSuQmCC
"
     style="display:block;margin:auto;width:10%"/>
<br>
<div style="text-align:center; font-size:200%;">
 <b>Generator-Funktionen und Coroutinen</b>
</div>
<br/>
<div style="text-align:center;">Dr. Matthias Hölzl</div>
<br/>
<!-- <div style="text-align:center;">module_130_functions/topic_370_e4_generator_functions</div> -->

## Generator Funktionen

Komplexere Fälle können von Generator Expressions nicht mehr abgedeckt werden.

- Generator, der alle Zahlen erzeugt (ohne Obergrenze)
- Generator, der ein Iterable modifiziert (z.B. mehrfach ausführt, eine fixe Anzahl
  an Elementen nimmt)

Für diese Fälle gibt es Generator-Funktionen


Mit dem Schlüsselwort `yield` kann eine Funktion "mehrere Werte zurückgeben".
Eine Funktion, die `yield` in ihrem Rumpf verwendet, wird Generatorfunktion
genannt. Ein Aufruf einer Generatorfunktion wertet nicht den Rumpf der
Funktion aus, sondern es wird ein *Generator* zurückgegeben, der mehrmals
einen Wert zurückgeben kann:

In [ ]:
def integers(start=0):
    n = start
    while True:
        yield n
        n += 1

In [ ]:
gen = integers()
print(repr(gen))
print(repr(iter(gen)))

In [ ]:
gen = integers()

In [ ]:
next(gen)

In [ ]:
next(gen)

In [ ]:
for i in integers():
    if i > 3:
        break
    print(i, end=" ")


## Mini-Workshop

Schreiben Sie eine Generatorfunktion `one_based_range(n)`, die `range(n)` für
ein Argument simuliert, aber von 1 bis einschließlich `n` iteriert.

In [ ]:
def one_based_range(n):
    for i in range(1, n + 1):
        yield i

In [ ]:
assert list(one_based_range(4)) == [1, 2, 3, 4]


Schreiben Sie eine Generatorfunktion `inklusive_range()`, die die komplette
Funktionalität von `range()` simuliert (d.h. die mit einem, zwei oder drei
Argumenten aufgerufen werden kann), aber ihre obere Grenze einschließt und die
Iteration von 1 beginnt, wenn keine untere Grenze angegeben wird.

Stellen Sie sicher, dass Ihre Implementierung die vorgegebenen Testfälle
erfüllt.

In [ ]:
from typing import overload

In [ ]:
@overload
def inclusive_range(end):
    ...


@overload
def inclusive_range(begin, end):
    ...

In [ ]:
def inclusive_range(begin, end=None, step=1):
    if end is None:
        end = begin
        begin = 1
    for i in range(begin, end + 1, step):
        yield i

In [ ]:
inclusive_range(1)

In [ ]:
assert list(inclusive_range(3)) == [1, 2, 3]

In [ ]:
assert list(inclusive_range(2, 4)) == [2, 3, 4]

In [ ]:
assert list(inclusive_range(2, 2)) == [2]

In [ ]:
assert list(inclusive_range(2, 1)) == []

In [ ]:
assert list(inclusive_range(2, 6, 2)) == [2, 4, 6]


Wir können Generatorfunktionen verwenden, um Funktionen zu schreiben, die
Iteratoren verarbeiten. Beispielsweise nimmt die Generatorfunktion `take()`
eine feste Anzahl von Werten von einem Iterator:

In [ ]:
def take(n, it):
    for i in range(n):
        yield next(it)

In [ ]:
list(take(3, integers()))


Beachten Sie, dass die Funktion `drop()`, die die ersten `n` Elemente eines
Iterators entfernt, keine Generatorfunktion, sondern eine reguläre Funktion
ist:

In [ ]:
def drop(n, it):
    for i in range(n):
        next(it)
    return it

In [ ]:
list(take(3, drop(2, integers())))


Mit Generatorfunktionen können wir auch komplexere Iterations-Operationen
definieren:

In [ ]:
def repeat_n_times(n, it):
    for _ in range(n):
        for elt in it:
            yield elt

In [ ]:
for num in repeat_n_times(3, range(5)):
    print(num, end=" ")

In [ ]:
def repeat_n_times_2(n, it):
    for _ in range(n):
        yield from it

In [ ]:
for num in repeat_n_times_2(3, range(5)):
    print(num, end=" ")


## Mini-Workshop "Generator"

Schreiben Sie einen Generator `leaves(tree: Bintree)`, der alle Blätter eines
Binärbaums zurückgibt (d.h., alle Knoten, die nicht Instanzen von `Bintree` sind).

In [ ]:

from dataclasses import dataclass
from typing import Optional

In [ ]:
@dataclass
class Bintree:
    value: object = None
    left: Optional["Bintree"] = None
    right: Optional["Bintree"] = None

In [ ]:
def leaves(tree: Bintree):
    if tree:
        yield from leaves(tree.left)
        yield tree.value
        yield from leaves(tree.right)

In [ ]:
tree1 = Bintree(1)
tree2 = Bintree(2, Bintree(1), Bintree(3))
tree3 = Bintree(
    1, Bintree(0), Bintree(3, Bintree(2), Bintree(5, Bintree(4), Bintree(6)))
)

In [ ]:
assert tuple(leaves(tree1)) == (1,)

In [ ]:
assert tuple(leaves(tree2)) == (1, 2, 3)

In [ ]:
assert tuple(leaves(tree3)) == tuple(range(7))


# Coroutinen

`yield` kann auch verwendet werden, um einen Wert an die Stelle zurückzugeben,
an der es "aufgerufen" wird. In diesem Fall nennen wir den Generator auch eine
*Coroutine*.

Coroutinen sind nützliche Bausteine für Features wie z.B. kooperatives
Multitasking oder Event-basierte Programmierung.

Um eine Coroutine `c` zu starten, rufen wir die Methode `c.send(None)` auf.
Zum Senden nachfolgender Werte verwenden wir `c.send(value)`.


In [ ]:
def my_coroutine(n):
    for i in range(n):
        x = yield
        print(x)

In [ ]:
c = my_coroutine(3)
print(c)

In [ ]:
# Initial value must be `None`
c.send(None)

In [ ]:
c.send(10)

In [ ]:
c.send(20)

In [ ]:
# c.send(40)

In [ ]:
def your_coroutine(n):
    for i in range(1, n + 1):
        x = yield i
        print("your_coroutine:", x)

In [ ]:
c = your_coroutine(3)

In [ ]:
_x = c.send(None)
print("top level:", _x)

In [ ]:
try:
    while True:
        _x = c.send(_x * 10)
        print("top level:", _x)
except StopIteration:
    print("Done.")

In [ ]:
def counter(max_value):
    i = 0
    while i < max_value:
        val = yield i
        if val is not None:
            i = val
        else:
            i += 1

In [ ]:
it = counter(100)

In [ ]:
next(it)

In [ ]:
next(it)

In [ ]:
it.send(98)

In [ ]:
next(it)

In [ ]:
# next(it)